<a target="_blank" href="https://colab.research.google.com/github/wakusoftware/intro_cuantizacion/blob/master/lab_3_cuantizacion_lineal_(opcional).ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# Lab 3: Cuantización Lineal en Pytorch (Opcional)

En esta sección, implementaremos la cuantización lineal para tensores de PyTorch.

## Introducción

La cuantización es una técnica utilizada para reducir la precisión de los números en los modelos de aprendizaje automático, con el objetivo de disminuir el uso de memoria y aumentar la velocidad de inferencia. La **cuantización lineal** es un método que mapea valores de punto flotante (fp32) a un rango más pequeño utilizando números enteros (int8). Este laboratorio te guiará a través de los conceptos básicos y te proporcionará ejemplos prácticos en PyTorch.

## Conceptos Básicos de la Cuantización Lineal

### ¿Qué es la Cuantización Lineal?

La cuantización lineal transforma los valores continuos (float32) en un conjunto discreto de valores (int8). Esta transformación se realiza mediante una operación matemática simple que involucra dos parámetros clave: **escala** y **punto cero**.

### Escala (Scale)

La **escala** es un factor que determina la resolución de la cuantización. Es como un "multiplicador" que ajusta el rango de los valores de punto flotante para que se puedan representar en el rango más pequeño de valores enteros.

- Imagina que tienes un termómetro que mide de 0 a 100 grados y quieres representar esa temperatura en una escala más pequeña de 0 a 10. La escala sería 10 (es decir, cada unidad en la nueva escala representa 10 grados en la escala original).

### Punto Cero (Zero Point)

El **punto cero** es un valor de desplazamiento que asegura que el rango de valores int8 mapea correctamente al rango de valores fp32.

- Siguiendo con el ejemplo del termómetro, si decides que el valor 0 en la nueva escala (0 a 10) debe corresponder a 30 grados en la escala original, el punto cero sería 3 (es decir, 30 grados menos la escala de 10).

En términos matemáticos, la cuantización se realiza con la siguiente fórmula:

\[ \text{valor\_cuantizado} = \text{round}(\text{valor\_flotante} / \text{escala}) + \text{punto\_cero} \]

Y la descuantización se realiza con:

\[ \text{valor\_flotante} = (\text{valor\_cuantizado} - \text{punto\_cero}) \times \text{escala} \]

### Beneficios de la Cuantización Lineal

- **Reducción del uso de memoria**: Los modelos cuantizados utilizan menos memoria.
- **Aumento de la velocidad**: Los cálculos en int8 son más rápidos en comparación con fp32.
- **Menor consumo de energía**: Los dispositivos de hardware optimizados para cuantización pueden consumir menos energía.


## Ejemplo Práctico en PyTorch

### Paso 1: Configuración del Entorno

Primero, asegúrate de tener los paquetes instalados. Puedes instalarlos así:

```bash
pip install -r requirements.txt
```

Si estas usando Google Colab, no es necesario instalar los paquetes ya que Colab ya los tiene instalados.

### Paso 2: Definir una Función de Cuantización Lineal
A continuación, definimos una función que realizará la cuantización lineal de un tensor de punto flotante a un tensor de enteros de 8 bits.



In [ ]:
import torch

def linear_quantize(tensor, scale, zero_point, dtype=torch.int8):
    """
    Aplica cuantización lineal a un tensor.

    Args:
    tensor (torch.Tensor): El tensor de punto flotante a cuantizar.
    scale (float): El factor de escala para la cuantización.
    zero_point (int): El punto cero para la cuantización.
    dtype (torch.dtype): El tipo de datos del tensor cuantizado.

    Returns:
    torch.Tensor: El tensor cuantizado.
    """
    quantized_tensor = torch.round(tensor / scale) + zero_point
    quantized_tensor = quantized_tensor.clamp(0, 255).to(dtype)
    return quantized_tensor


### Paso 3: Cuantización y Descuantización de un Tensor
En este ejemplo, cuantizaremos y luego descuantizaremos un tensor de muestra

In [ ]:
# Crear un tensor de muestra
tensor_fp32 = torch.tensor([1.0, 2.5, 3.0, 4.5, 5.0], dtype=torch.float32)

# Definir los parámetros de cuantización
scale = 0.1
zero_point = 128

# Aplicar la cuantización lineal
tensor_int8 = linear_quantize(tensor_fp32, scale, zero_point)

print("Tensor cuantizado (int8):", tensor_int8)

# Función de descuantización
def linear_dequantize(tensor, scale, zero_point):
    """
    Aplica descuantización lineal a un tensor.

    Args:
    tensor (torch.Tensor): El tensor cuantizado.
    scale (float): El factor de escala utilizado para la cuantización.
    zero_point (int): El punto cero utilizado para la cuantización.

    Returns:
    torch.Tensor: El tensor descuantizado.
    """
    dequantized_tensor = (tensor.to(torch.float32) - zero_point) * scale
    return dequantized_tensor

# Aplicar la descuantización lineal
tensor_dequantized = linear_dequantize(tensor_int8, scale, zero_point)

print("Tensor descuantizado (fp32):", tensor_dequantized)


### Explicación del Código

- Función de Cuantización Lineal: La función linear_quantize toma un tensor de punto flotante, un factor de escala, un punto cero y devuelve un tensor cuantizado en int8.
-
- Cuantización del Tensor: El tensor de punto flotante tensor_fp32 se cuantiza utilizando la función linear_quantize.
- Función de Descuantización Lineal: La función linear_dequantize toma un tensor cuantizado y lo convierte de nuevo a un tensor de punto flotante utilizando los mismos parámetros de escala y punto cero.
- Descuantización del Tensor: El tensor cuantizado tensor_int8 se descuantiza utilizando la función linear_dequantize.


## Diferencias entre Cuantización Simétrica y Asimétrica

### Cuantización Simétrica

En la **cuantización simétrica**, el rango de los valores de punto flotante se mapea de manera uniforme alrededor del cero. Esto significa que el valor cero en la representación cuantizada corresponde al valor cero en la representación de punto flotante. La fórmula para la cuantización simétrica es:

\[ \text{valor\_cuantizado} = \text{round}(\text{valor\_flotante} / \text{escala}) \]

- **Escala (scale)**: Es un factor que determina la resolución de la cuantización. En cuantización simétrica, la escala es el mismo factor tanto para los valores positivos como para los negativos.
- **Punto cero (zero point)**: En cuantización simétrica, el punto cero es siempre 0.

**Ventajas**:
- La implementación es más sencilla, ya que no hay desplazamiento adicional.
- Es adecuada cuando los datos de entrada están centrados alrededor de cero.

**Desventajas**:
- No es eficiente si los datos tienen un rango asimétrico, ya que puede desperdiciar parte del rango de representación.

### Cuantización Asimétrica

En la **cuantización asimétrica**, el rango de los valores de punto flotante se mapea de manera que el punto cero puede no ser cero. Esto permite un mejor aprovechamiento del rango de representación, especialmente cuando los datos no están centrados alrededor de cero. La fórmula para la cuantización asimétrica es:

\[ \text{valor\_cuantizado} = \text{round}(\text{valor\_flotante} / \text{escala}) + \text{punto\_cero} \]

- **Escala (scale)**: Determina la resolución de la cuantización. En cuantización asimétrica, puede ser diferente para los valores positivos y negativos.
- **Punto cero (zero point)**: Es un desplazamiento que asegura que el rango de valores cuantizados mapea correctamente al rango de valores de punto flotante.

**Ventajas**:
- Mejor utilización del rango de representación, especialmente para datos con distribución asimétrica.
- Puede reducir el error de cuantización en modelos donde los datos de entrada no están centrados alrededor de cero.

- En este laboratorio, hemos utilizado cuantización lineal asimétrica, ya que incluimos tanto la escala como el punto cero en la fórmula de cuantización:

\[ \text{valor\_cuantizado} = \text{round}(\text{valor\_flotante} / \text{escala}) + \text{punto\_cero} \]
